# Oanda Demo Trading Notebook

## Packages

Normal Packages

In [4]:
import numpy as np
import pandas as pd

import yaml
import json

import time
import datetime
import winsound
import collections

from tqdm import tqdm

import warnings
warnings.filterwarnings('ignore')

Oanda Packages

In [19]:
from oandapyV20 import API
import oandapyV20.endpoints.trades as trades
import oandapyV20.endpoints.pricing as pricing
import oandapyV20.endpoints.accounts as accounts

import oandapyV20.definitions.pricing as defpricing
import oandapyV20.endpoints.instruments as instruments

import oandapyV20.endpoints.orders as orders

from oandapyV20.contrib.requests import (MarketOrderRequest, StopLossDetails)
import oandapyV20.endpoints.positions as positions

Custom packages

In [6]:
from utils.toolsapi import get_date_time
from utils.toolsapi import get_prices
from utils.toolsapi import calc_duration

## Functions

In [4]:
def get_targets(start_price, target_num = 3):
    positive_targets = {}
    negative_targets = {}
    pip = 10**-4
    
    for i in range(target_num):
        move_val = pip*(i+1)
        positive_targets[i] = start_price + move_val
        negative_targets[i] = start_price - move_val
        
    return(positive_targets, negative_targets)

In [10]:
def get_direction(df_reached_targets, target_num,  min_count = 10):
    tot_pos = sum(df_reached_targets['positive'] * df_reached_targets['target_num'])
    tot_neg = sum(df_reached_targets['negative'] * df_reached_targets['target_num'])

    if ((tot_pos - tot_neg) / target_num) > 1 and tot_pos > min_count and df_reached_targets['positive'][1] > 0:
        direction = 'positive'
    elif ((tot_neg - tot_pos) / target_num) > 1 and tot_neg > min_count and df_reached_targets['negative'][1] > 0:
        direction = 'negative'
    else:
        direction = 'no_direction'
    return(direction)

In [6]:
def run_engine(params, num, target_num, min_count): 
    
    reached_targets = {'start_price' : 0,
                       'target_num' : 0,
                       'positive_val' : 0,
                       'negative_val' : 0,
                       'positive' : 0,
                       'p_duration' : '',
                       'n_duration' : '',
                       'negative' : 0}

    pos_target_flag = 'not_reached'
    neg_target_flag = 'not_reached'
    first_run_flag = 0
    df_reached_targets = pd.DataFrame()
    tick_list = []

    
    
    r = pricing.PricingStream(accountID=accountID, params=params)
    rv = api.request(r)

    start_time = time.time()

    for i, resp in tqdm(enumerate(rv)):

        if i < num: # Check if we are within the required number of price iterations               
            resp_type = resp['type']       

            if resp_type == 'HEARTBEAT': # Heart beat response to keep the api connection alive (Avoid timeout)
                pass
                #print(resp_type)

            elif resp_type == 'PRICE': # Check whether it is a price response                 
                date_val, time_val, time_fraction = get_date_time(resp) # Get time stamp for reference            
                sell_price, buy_price, spread, tick_price = get_prices(resp) # Get prices from the response                      
                tick_list.append(tick_price)

                if first_run_flag == 0:
                    positive_targets, negative_targets = get_targets(tick_price, target_num)
                    first_run_flag = 1
                    for j in range(target_num):
                        df_reached_targets = df_reached_targets.append(reached_targets, ignore_index = True)
                        df_reached_targets.loc[df_reached_targets.index[j], 'target_num'] = j+1
                        df_reached_targets.loc[df_reached_targets.index[j], 'start_price'] = tick_price
                        df_reached_targets.loc[df_reached_targets.index[j], 'positive_val'] = positive_targets[j]
                        df_reached_targets.loc[df_reached_targets.index[j], 'negative_val'] = negative_targets[j]

                for k in range(target_num):
                    if tick_price >= positive_targets[k]:
                        df_reached_targets.loc[df_reached_targets.index[k], 'positive'] += 1
                        if pos_target_flag == 'not_reached':
                            end_time = time.time()    
                            duration = calc_duration(start_time, end_time)
                            df_reached_targets.loc[df_reached_targets.index[k], 'p_duration'] = duration
                            pos_target_flag = 'reached'

                    if tick_price <= negative_targets[k]:
                        df_reached_targets.loc[df_reached_targets.index[k], 'negative'] += 1            
                        if neg_target_flag == 'not_reached':
                            end_time = time.time()    
                            duration = calc_duration(start_time, end_time)
                            df_reached_targets.loc[df_reached_targets.index[k], 'n_duration'] = duration
                            neg_target_flag = 'reached'

        else: # Crossed the required number of price iterations
            try:
                r.terminate(message = "maxrecs records received")
            except:
                pass

    df_reached_targets =  df_reached_targets[['start_price', 'target_num', 'positive_val', 'negative_val','positive', 'negative','p_duration','n_duration']]
    direction = get_direction(df_reached_targets, target_num, min_count)

    #winsound.PlaySound('C:\\Windows\\Media\\tada.wav', winsound.SND_ASYNC) 
    return(direction, df_reached_targets, tick_list)

## API Setup

Read from config file

In [8]:
config_file = 'config/access_token.yaml'

with open(config_file) as c_file:
    config = yaml.load(c_file)

access_token = config['oanda_demo_account']['token']
accountID = config['oanda_demo_account']['account_id']

api = API(access_token = access_token)

## Code Engine

In [11]:
params = {
    #'instruments': 'EUR_USD'
    'instruments': 'AUD_SGD'
}

num = 3 * 60
target_num = 3
min_count = 10

direction, df_reached_targets, tick_list = run_engine(params, num, target_num, min_count)
print(f'direction : {direction}')
df_reached_targets

181it [05:17,  1.76s/it]

direction : negative


,start_price,target_num,positive_val,negative_val,positive,negative,p_duration,n_duration
0,0.95263,1.0,0.95273,0.95253,0.0,106.0,,0:0:21
1,0.95263,2.0,0.95283,0.95243,0.0,95.0,,
2,0.95263,3.0,0.95293,0.95233,0.0,84.0,,


In [13]:
stopLossOnFill = StopLossDetails(price=1.1311)

ordr = MarketOrderRequest(
    instrument="EUR_USD",
    units=1,
    stopLossOnFill=stopLossOnFill.data)

r = orders.OrderCreate(accountID, data=ordr.data)
rv = api.request(r)

In [30]:
def make_order(accountID, stop_price, instrument, units):
    stopLossOnFill = StopLossDetails(price=stop_price)

    ordr = MarketOrderRequest(
        instrument = instrument,
        units=units,
        stopLossOnFill=stopLossOnFill.data)

    r = orders.OrderCreate(accountID, data=ordr.data)
    rv = api.request(r)
    return(rv)

In [31]:
def close_order(accountID, order_type, instrument):
    data_long = {"longUnits": "ALL"}
    data_short = {"longUnits": "ALL"}
    
    if order_type == 'long':
        data = data_long
    elif order_type == 'short':
        data = data_short
        
    r = positions.PositionClose(accountID=accountID,
                                instrument=instrument,
                                data=data)
    rv = api.request(r)
    return(rv)

In [38]:
instrument="EUR_USD"
units=1
stop_price = 1.1315
order_type = 'long'

In [39]:
make_order_log = make_order(accountID, stop_price, instrument, units)

In [41]:
close_order_log = close_order(accountID, order_type, instrument)

V20Error: {"longOrderRejectTransaction":{"type":"MARKET_ORDER_REJECT","rejectReason":"CLOSEOUT_POSITION_DOESNT_EXIST","instrument":"EUR_USD","timeInForce":"FOK","positionFill":"REDUCE_ONLY","reason":"POSITION_CLOSEOUT","longPositionCloseout":{"instrument":"EUR_USD","units":"ALL"},"id":"255","accountID":"101-003-15069707-001","userID":15069707,"batchID":"255","requestID":"24691552593050375","time":"2020-06-12T10:26:29.110014177Z"},"relatedTransactionIDs":["255"],"lastTransactionID":"255","errorMessage":"The Position requested to be closed out does not exist","errorCode":"CLOSEOUT_POSITION_DOESNT_EXIST"}